In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('G20-Cleansing.csv')

In [4]:
from nltk.tokenize import word_tokenize 

word_dict = {}
for i in range(0,len(df['tweet'])):
    sentence = df['tweet'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in word_dict:
            word_dict[j] = 1
        else:
            word_dict[j] += 1

In [5]:
# import kamus lexicon dan menghapus kata negasi

negasi = ['bukan','tidak','ga','gk']
lexicon = pd.read_csv('modified_full_lexicon.csv')
# lexicon = lexicon.drop(lexicon[(lexicon['word'] == 'bukan')
#                               |(lexicon['word'] == 'tidak')
#                               |(lexicon['word'] == 'ga')|(lexicon['word'] == 'gk') ].index,axis=0)
lexicon = lexicon.reset_index(drop=True)

In [6]:
lexicon_word = lexicon['word'].to_list()
lexicon_num_words = lexicon['number_of_words']

In [7]:
# memeriksa kata dalam kamus yang tidak termasuk dalam lexicon
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

ns_words = []
factory = StemmerFactory()
stemmer = factory.create_stemmer()
for word in word_dict.keys():
    if word not in lexicon_word:
        kata_dasar = stemmer.stem(word)
        if kata_dasar not in lexicon_word:
            ns_words.append(word)
len(ns_words)

3240

In [8]:
sencol =[]
senrow =np.array([])
nsen = 0
factory = StemmerFactory()
stemmer = factory.create_stemmer()
sentiment_list = []
# function to write the word's sentiment if it is founded
def found_word(ind,words,word,sen,sencol,sentiment,add):
    # if it is already included in the bag of words matrix, then just increase the value
    if word in sencol:
        sen[sencol.index(word)] += 1
    else:
    #if not, than add new word
        sencol.append(word)
        sen.append(1)
        add += 1
    #if there is a negation word before it, the sentiment would be the negation of it's sentiment
    if (words[ind-1] in negasi):
        sentiment += -lexicon['weight'][lexicon_word.index(word)]
    else:
        sentiment += lexicon['weight'][lexicon_word.index(word)]
    
    return sen,sencol,sentiment,add
            
# checking every words, if they are appear in the lexicon, and then calculate their sentiment if they do
for i in range(len(df)):
    nsen = senrow.shape[0]
    words = word_tokenize(df['tweet'][i])
    sentiment = 0 
    add = 0
    prev = [0 for ii in range(len(words))]
    n_words = len(words)
    if len(sencol)>0:
        sen =[0 for j in range(len(sencol))]
    else:
        sen =[]
    
    for word in words:
        ind = words.index(word)
        # check whether they are included in the lexicon
        if word in lexicon_word :
            sen,sencol,sentiment,add= found_word(ind,words,word,sen,sencol,sentiment,add)
        else:
        # if not, then check the root word
            kata_dasar = stemmer.stem(word)
            if kata_dasar in lexicon_word:
                sen,sencol,sentiment,add= found_word(ind,words,kata_dasar,sen,sencol,sentiment,add)
        # if still negative, try to match the combination of words with the adjacent words
            elif(n_words>1):
                if ind-1>-1:
                    back_1    = words[ind-1]+' '+word
                    if (back_1 in lexicon_word):
                        sen,sencol,sentiment,add= found_word(ind,words,back_1,sen,sencol,sentiment,add)
                    elif(ind-2>-1):
                        back_2    = words[ind-2]+' '+back_1
                        if back_2 in lexicon_word:
                            sen,sencol,sentiment,add= found_word(ind,words,back_2,sen,sencol,sentiment,add)
    # if there is new word founded, then expand the matrix
    if add>0:  
        if i>0:
            if (nsen==0):
                senrow = np.zeros([i,add],dtype=int)
            elif(i!=nsen):
                padding_h = np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding_h))
                padding_v = np.zeros([(i-nsen),senrow.shape[1]],dtype=int)
                senrow = np.vstack((senrow,padding_v))
            else:
                padding =np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding))
            senrow = np.vstack((senrow,sen))
        if i==0:
            senrow = np.array(sen).reshape(1,len(sen))
    # if there isn't then just update the old matrix
    elif(nsen>0):
        senrow = np.vstack((senrow,sen))
        
    sentiment_list.append(sentiment)

In [9]:
# membangun data frame yang berisi bag of words dan sentimen yang dihitung sebelumnya

sencol.append('sentiment')
sentiment_array = np.array(sentiment_list).reshape(senrow.shape[0],1)
sentiment_data = np.hstack((senrow,sentiment_array))
df_sen = pd.DataFrame(sentiment_data,columns = sencol)

In [10]:
df_sen

,halo,ambil,harap,dunia,isu,habis,siap,rangkai,temu,keluar,...,amin,nyet,esok,murah,peduli,uji,beban,cair,liang,sentiment
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
1,0,0,0,0,0,1,3,1,1,1,...,0,0,0,0,0,0,0,0,0,7
2,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-4
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1749,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-3
1750,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
1751,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1752,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


In [11]:
# melihat sentimen dari tweet original

cek_df = pd.DataFrame([])
cek_df['created_at'] = df['created_at'].copy()
cek_df['tweet'] = df['tweet'].copy()
cek_df['sentiment']  = df_sen['sentiment'].copy()

In [12]:
cek_df

,created_at,tweet,sentiment
0,11/14/2022 21:39,halo indonesia bal ambil helat ktt g harap dis...,-1
1,11/14/2022 19:35,g rp miliar habis siap laksana rangkai temu g ...,7
2,11/15/2022 8:03,indonesia pegang ketua g desember posisi presi...,15
3,11/16/2022 10:07,bem mpm sejayapura tolak ktt g,-4
4,11/16/2022 10:07,polri patroli tebal aman acara puncak ktt g do...,0
...,...,...,...
1749,11/14/2022 17:16,bodoh forum g,-3
1750,11/14/2022 17:16,lampung dukung acara g lancar aman,8
1751,11/14/2022 17:16,presidensi g indonesia hasil concreate deliver...,3
1752,11/14/2022 17:16,indonesia tuan rumah ktt g gelar november bal ...,9


In [13]:
# create a list of our conditions
conditions = [
    (cek_df['sentiment'] < 0),
    (cek_df['sentiment'] == 0),
    (cek_df['sentiment'] > 0),
    ]

# create a list of the values we want to assign for each condition
values = ['negatif', 'netral', 'positif']

# create a new column and use np.select to assign values to it using our lists as arguments
cek_df['label'] = np.select(conditions, values)

In [14]:
cek_df

,created_at,tweet,sentiment,label
0,11/14/2022 21:39,halo indonesia bal ambil helat ktt g harap dis...,-1,negatif
1,11/14/2022 19:35,g rp miliar habis siap laksana rangkai temu g ...,7,positif
2,11/15/2022 8:03,indonesia pegang ketua g desember posisi presi...,15,positif
3,11/16/2022 10:07,bem mpm sejayapura tolak ktt g,-4,negatif
4,11/16/2022 10:07,polri patroli tebal aman acara puncak ktt g do...,0,netral
...,...,...,...,...
1749,11/14/2022 17:16,bodoh forum g,-3,negatif
1750,11/14/2022 17:16,lampung dukung acara g lancar aman,8,positif
1751,11/14/2022 17:16,presidensi g indonesia hasil concreate deliver...,3,positif
1752,11/14/2022 17:16,indonesia tuan rumah ktt g gelar november bal ...,9,positif


In [15]:
cek_df.to_csv("Sentimen-G20.csv")